In [1]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [2]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# tox-to-reproduction-header
tox_to_reproduct_path = '//*[@id="QU.SE.7.8-toxicity-to-reproduction-header"]/div/div[2]'
driver.find_element_by_xpath(tox_to_reproduct_path).click()
time.sleep(0.5)

# header 하위 3개의 창
tox_to_repro_path = '//*[@id="QU.SE.7.8-toxicity-to-reproduction"]/div/div/div[1]/div[3]/button'
driver.find_element_by_xpath(tox_to_repro_path).click()
time.sleep(0.5)

/tmp/ipykernel_103/686095874.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [3]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG416경로, 이건 바꿔야함
tg416_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[15]'
driver.find_element_by_xpath(tg416_path).click()
time.sleep(0.5)

#EPA8703800경로, 이것도 바꿔야함
epa8703800_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[4]'
driver.find_element_by_xpath(epa8703800_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [4]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [5]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).click()
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg416_source.json', 'w'))

 80%|█████████████████████████████████████████████████████████████▌               | 4/5 [00:16<00:04,  4.05s/it, page=8]


ElementClickInterceptedException: Message: element click intercepted: Element <button class="btn primaryBTN disabled btn-default" id="_viewsubstances_WAR_echarevsubstanceportlet_acceptDisclaimerButton" name="_viewsubstances_WAR_echarevsubstanceportlet_acceptDisclaimerButton" type="button">...</button> is not clickable at point (563, 526). Other element would receive the click: <div class="buttonContainer">...</div>
  (Session info: headless chrome=105.0.5195.125)


In [ ]:
##json전처리

In [9]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg416_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string


#%%
result_ = []
reproductive_effects = np.nan
reproductive_performance = np.nan

for i in tqdm(range(len(df))):
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn
        
        
        # clinical sign
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sResultsAndDiscussion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Clinical signs:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Clinical signs'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Clinical signs'] = np.nan
            
        # Reproductive effects observed1
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sResultsAndDiscussion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Reproductive effects observed:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Reproductive effects observed1'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Reproductive effects observed1'] = np.nan   
            
            
   
        # Reproductive effects observed2
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Reproductive effects observed" in str(admin_tmp[i]):
                    reproductive_effects = str(admin_tmp[i]).split('<dt>Reproductive effects observed:</dt><dd>')[1]
                    reproductive_effects = reproductive_effects.split('</dd>')[0]
                chem_dict['Reproductive effects observed2'] = reproductive_effects
        except ValueError:
            chem_dict['Reproductive effects observed2'] = np.nan
            
        # Reproductive performance:
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Reproductive performance" in str(admin_tmp[i]):
                    reproductive_performance = str(admin_tmp[i]).split('<dt>Reproductive performance:</dt><dd>')[1]
                    reproductive_performance = reproductive_performance.split('</dd>')[0]
                chem_dict['Reproductive performance'] = reproductive_performance
        except ValueError:
            chem_dict['Reproductive performance'] = np.nan
            
            
        # other results
        result_and_discussion = soup.find('h3', attrs={'id': 'sResultsAndDiscussion'})
        table_list = result_and_discussion.find_next_sibling('div').find_all('dl')
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        pass
    


# %%
result = pd.DataFrame(result_)

100%|█████████████████████████████████████████████████████████████████████████████████| 894/894 [00:51<00:00, 17.43it/s]


In [24]:
result['Reproductive effects observed2'].value_counts()

not specified    1059
no                406
yes               135
Name: Reproductive effects observed2, dtype: int64

In [25]:
result['Reproductive performance'].value_counts()

no effects observed                        1163
effects observed, treatment-related         332
effects observed, non-treatment-related      52
not examined                                 32
not specified                                21
Name: Reproductive performance, dtype: int64

In [13]:
result.to_excel('tg416_raw.xlsx', header = True, index = False)

In [29]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
result=pd.read_excel('tg416_0raw.xlsx')

In [30]:

#%%
'''
    1. effects observed가 하나라도 있으면 1
    1-1. no effects observed가 있으면 0
    
'''

def find_colon(string):
    return ':' in string


def check_nan(string):
    return string == string
        

# result['Route of administration'].unique()
# result['Route of administration'].value_counts()

# effect 항목의 결과 내용 중 포함이 될 경우
def effective(string):
    if check_nan(string):
        string = string.lower()
        if 'effects observed' in string and 'no effects observed' not in string:
            type_ = 1
            
        elif 'yes' in string:
            type_ = 1
        
        elif 'no' in string and 'not' not in string:
            type_ = 0
            
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

#%%
df = result.copy()

# effect
df['Reproductive effects observed1'] = df['Reproductive effects observed1'].map(lambda x: effective(x))
df['Reproductive effects observed2'] = df['Reproductive effects observed2'].map(lambda x: effective(x))
df['Reproductive performance'] = df['Reproductive performance'].map(lambda x: effective(x))

df = df[['CasRN', 'Reproductive effects observed1', 'Reproductive effects observed2', 'Reproductive performance']]

In [31]:
df['Reproductive performance'].value_counts()

0.0    1163
1.0     384
Name: Reproductive performance, dtype: int64

In [32]:
df['Reproductive effects observed2'].value_counts()

0.0    406
1.0    135
Name: Reproductive effects observed2, dtype: int64

In [33]:
# conclude target
df_null = df.isnull()
for i in range(len(df)):
    if df_null['Reproductive effects observed1'][i] == False:
        target = df['Reproductive effects observed1'][i]
    elif df_null['Reproductive effects observed2'][i] == False:
        target = df['Reproductive effects observed2'][i]
    elif df_null['Reproductive performance'][i] == False:
        target = df['Reproductive performance'][i]
    else : target = np.nan
    df.loc[i,'target'] = target

In [34]:
df_2nd=df.copy()
df_2nd['target'].value_counts()

0.0    1192
1.0     372
Name: target, dtype: int64

In [36]:
df_2nd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CasRN                           1600 non-null   object 
 1   Reproductive effects observed1  0 non-null      float64
 2   Reproductive effects observed2  541 non-null    float64
 3   Reproductive performance        1547 non-null   float64
 4   target                          1564 non-null   float64
dtypes: float64(4), object(1)
memory usage: 62.6+ KB


In [37]:
# save df
df.to_excel('tg416_pre1.xlsx', header = True, index = False)

# smiles 추가

In [38]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [39]:
data = pd.read_excel('tg416_pre1.xlsx')
len(data['CasRN'].unique())

603

In [40]:
data.head()

CasRN  Reproductive effects observed1  Reproductive effects observed2  \
0  55406-53-6                             NaN                             0.0   
1  55406-53-6                             NaN                             0.0   
2  55406-53-6                             NaN                             0.0   
3  55406-53-6                             NaN                             0.0   
4  13674-84-5                             NaN                             NaN   

   Reproductive performance  target  
0                       0.0     0.0  
1                       0.0     0.0  
2                       0.0     0.0  
3                       0.0     0.0  
4                       0.0     0.0

In [41]:
data['target'].unique()
data['target'].isna().sum()

data = data[data['target'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['target'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [42]:
data = data.drop_duplicates()
len(data)

586

In [43]:
data1 = data[:300]
data2 = data[300:]

In [44]:
tqdm.pandas()
data1['SMILES'] = data1.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [05:26<00:00,  1.09s/it]


In [45]:
tqdm.pandas()
data2['SMILES'] = data2.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 286/286 [05:04<00:00,  1.06s/it]


In [46]:
data = pd.concat([data1, data2])

In [47]:
data.to_csv('tg416_2smiles.csv', header = True, index = False)

In [48]:
data.SMILES.isna().sum()
data = data[data['SMILES'].notna()].reset_index(drop = True)

In [49]:
data.to_csv('tg416_3dropnull.csv', header = True, index = False)